# `beerpong.py`

This is a solution to the following problem in [combinatorics](https://en.wikipedia.org/wiki/Combinatorics) and probablility.

<blockquote>In a <a href="https://en.wikipedia.org/wiki/Beer_pong">beer pong</a> tournament where pairs are chosen at random... and there are 126 participants and 12 are from New York... what are the chances there is at least one pair consisting of two New Yorkers?</blockquote>

## Solution

Calculate the number of `pairs`, and the number of `qualifying` pairs and take the ratio. To find the `qualifying` pairs, when looking for '*at least one*,' look for '*none*' (the `nonqualifying` pairs) and subtract.

Assuming `total` people is a multiple of $2$, so they can be paired, and `special` people is $<= total \div 2$, because (by the [Pigeon Hole Principle](https://en.wikipedia.org/wiki/Pigeonhole_principle)) the probability is $100\%$ for $special > total \div 2$ and given `total` $t$ and `special` $s$:
- the number of `total` pairs is $t! \div \left(2!\right)^{\frac{t}{2}}$ &mdash; permuting the `total` people and dividing by $\left(2!\right)^{\frac{t}{2}}$ because the order of each of the $\frac{t}{2}$ `total` pairs doesn't matter;
- the number of `nonqualifying` pairs is $\binom{\frac{t}{2}}{s} \times s! \times (t - s)! \div \left(2!\right)^{\left( \frac{t}{2} - s \right)}$ &mdash; choosing the pairs for the unpaired `special` people, permuting the `special` people, permuting the non-`special` people, and dividing by $\left(2!\right)^{\left( \frac{t}{2} - s \right)}$ because the order of each of the $\left( \frac{t}{2} - s \right)$ pairs of non-`special` people doesn't matter.

The probability is, therefore, $1 - (\texttt{nonqualifying}) \div (\texttt{total})$.

So for this specific problem:

$$
\boxed{1 - \left[\frac{\binom{63}{12} \times 12! \times 114! \div \left(2!\right)^{51}}{126! \div \left(2!\right)^{63}}\right] = 43.86\%}
$$

## Solve it twice

I have always found that, the best way to understand a problem in combinatorics, is to solve it in two different ways. This problem can be solved by calculating the `qualifying` pairs as well as the `nonqualifying` pairs &mdash; which must, of course, sum to the `total` pairs.

Calculating '*at least one*' (the `qualifying` pairs) is more difficult than calculating '*none*' (the `nonqualifying` pairs) because each configuration with *at least one* `special` pair must be summed as follows:

$$
\sum_{i = 1}^{i < \frac{s}{2}} \underbrace{\binom{\frac{t}{2}}{i}}_{\text{paired NYers}} \times \underbrace{\binom{\frac{t}{2} - i}{s - 2i}}_{\text{unpaired NYers}} \times \underbrace{\binom{\left( \frac{t}{2} - i \right) - \left(s - 2i \right)}{\frac{t}{2} - s + i}}_{\text{paired non-NYers}} \times \left( \frac{s!}{\left(2!\right)^{i}} \right) \times \left( \frac{\left( t - s \right)!}{\left(2!\right)^{\left( \frac{t}{2} - s + i \right)}} \right)
$$
Note: $\underbrace{\binom{\left( \frac{t}{2} - i \right) - \left(s - 2i \right)}{\frac{t}{2} - s + i}}_{\text{paired non-NYers}} = \binom{n}{n} = 1$, because there is only one way to choose the paired non-New Yorkers.

For example, for $t = 12$ & $s = 5$, where `total` $= 12! \div \left( 2! \right)^{6} = 30656102400 \div 64 = 7484400$, there are `qualifying` configurations with $1$ pair and with $2$ pairs, as follows:
<table style="width: 100%;">
  <tr>
    <th style="text-align: center;">Paired `special`</th>
    <th style="text-align: center;">Sample Layout</th>
    <th style="text-align: center; width: 50%;">Total</th>
  </tr>
  <tr>
    <td style="text-align: center;">1</td>
    <td style="text-align: center;">`NN NX NX NX XX XX`</td>
    <td style="text-align: center;">$\binom{6}{1} \binom{5}{3} \binom{2}{2} \frac{5!}{\left(2!\right)^{1}} \frac{7!}{\left(2!\right)^{2}} = 6 \times 10 \times 1 \times 120 \div 2 \times 5040 \div 4 = 4536000$</td>
  </tr>
  <tr>
    <td style="text-align: center;">2</td>
    <td style="text-align: center;">`NN NN NX XX XX XX`</td>
    <td style="text-align: center;">$\binom{6}{2} \binom{4}{1} \binom{3}{3} \frac{5!}{\left(2!\right)^{2}} \frac{7!}{\left(2!\right)^{3}} = 15 \times 4 \times 1 \times 120 \div 4 \times 5040 \div 8 = 1134000$</td>
  </tr>
</table>

And&hellip; $4536000 + 1134000 = 5670000 = \underbrace{\left(12! \div \left(2!\right)^{6}\right) - \left( \binom{6}{5} \times 5! \times 7! \div \left(2!\right)^{1}\right)}_{\text{calculated the original way}} = 7484400 - 1814400$

So&hellip; pairing $12$ people, where $5$ of them are New Yorkers, into $6$ pairs, the probability is $5670000 \div 7484400 = 75.76\%$ that there will be *at least one* pair with two New Yorkers.

In [1]:
#!/usr/bin/env python3
#
# beerpong.py
#

import itertools, math, os, sys


def binomial(n, k):
    """Return n choose k."""
    assert n >= k >= 0, \
        'n ({}) choose k ({}) parameter error'.format(n, k)
    return math.factorial(n) // math.factorial(n - k) // math.factorial(k)


def is_unqualified(l, extra):
    """Return True if no pairs in l both have extra, False otherwise.
    Pairs are l[2 * n] and l[2 * n + 1]. len(l) must be even."""
    assert len(l) % 2 == 0, 'len({}) must be even'.format(l)
    for i in range(0, len(l), 2):
        if extra in l[i] and extra in l[i + 1]:
            return False
    return True


def verify(total, special, nonqualifying, maximum=8, extra='N'):
    """Verify formula(s) by generating actual nonqualified permutations,
    when total <= maximum, and comparing to nonqualifying result."""
    if total <= maximum:
        two_to_the_half = 2 ** (total // 2)
        people, nonqualified = [str(x) for x in range(total)], list()
        for i in range(special):
            people[i] = people[i] + extra
        nonqualified = [x for x in itertools.permutations(people)
                        if is_unqualified(x, extra)]
        result = len(nonqualified) // two_to_the_half
        assert result == nonqualifying, \
            '{} / {} = {} != {}' \
            .format(len(nonqualified), two_to_the_half, result, nonqualifying)


def beerpong(total, special):
    """To solve the following combinatorics problem:

        In a beer pong tournament where pairs are chosen at random...
        and there are 126 participants and 12 are from New York...
        what are the chances there is at least one pair consisting of
        two New Yorkers?

    return total pairs, qualifying pairs (2 NYers), and non-qualifying pairs
    (0 or 1 NYers)."""

    assert special <= total / 2, 'special ({}) > total ({}) / 2 ({})' \
        .format(special, total, total / 2)
    assert total % 2 == 0, 'total ({}) must be even'.format(total)

    half, diff = total // 2, total - special
    two_to_the_half, two_to_the_rest = 2 ** half, 2 ** (half - special)

    # Calculate total pairs and unqualified pairs.
    pairs, qual, unqual = math.factorial(total) // two_to_the_half, 0, \
        binomial(half, special) * math.factorial(special) \
        * math.factorial(diff) // two_to_the_rest

    verify(total, special, unqual)  # verify by generating actual permutations

    # Calculate qualified pairs.
    for i in range(1, special // 2 + 1):
        """
        print(tot, special, i, special - 2 * i)
        print(f"qual += "
              f"{binomial(half, i)} * {binomial(half - i, special - 2 * i)} * "
              f"binomial({half - i - (special - 2 * i)}, "
              f"{half - special + i}) * "
              f"{math.factorial(special)} // {2 ** i} * "
              f"{math.factorial(tot - special)} // {2 ** (half - i)}")
        """
        qual += binomial(half, i) * binomial(half - i, special - 2 * i) \
            * binomial(half - i - (special - 2 * i), half - special + i) \
            * math.factorial(special) // 2 ** i \
            * math.factorial(total - special) // 2 ** (half - special + i)
    assert qual == pairs - unqual, \
        f"{qual} != {pairs} - {unqual} = {pairs - unqual}"

    return pairs, qual, unqual


def test():
    """Test beerpong, including solution to beerpong(126, 12)."""

    # Print solution to beerpong(126, 12).
    i, j = 126, 12
    p, q, u = beerpong(i, j)
    assert q + u == p, f"{n} + {q} != {p}"
    print(f"{i:3d} {j:2d} {(1 - u / p) * 100:6.2f}% = 1 - ({u} \u00f7 {p})")
    """
    # Test code for listing intermediate test results.
    tot, spl, c = 6, 2, 'N'
    people = [str(x) for x in range(tot)]
    for i in range(spl):
        people[i] = people[i] + c

    # Actually generate the permutations to check.
    perm = list(itertools.permutations(people))
    qual = [x for x in perm if not is_unqualified(x, c)]
    unqual = [x for x in perm if is_unqualified(x, c)]
    print(f"{tot}! = {len(perm)} = {len(unqual)} + {len(qual)}")
    print(f"{len(qual)} / {len(perm)} = {len(qual) / len(perm)}")
    print(f"some: {len(qual)} {qual}")
    print(f"none: {len(unqual)} {unqual}")
    """
    # Print solutions for all beerpong problems on [0, 126].
    for i in range(0, 126 + 1, 2):
        for j in range(i // 2 + 1):
            p, q, u = beerpong(i, j)
            assert q + u == p, f"{n} + {q} != {p}"
            print(f"{i:3d} {j:2d} {(1 - u / p) * 100:6.2f}% = "
                  f"1 - ({u} \u00f7 {p})")


if __name__ == '__main__':
    is_idle, is_pycharm, is_jupyter = (
        'idlelib' in sys.modules,
        int(os.getenv('PYCHARM', 0)),
        '__file__' not in globals()
        )
    if is_idle or is_pycharm or is_jupyter:
        test()


126 12  43.86% = 1 - (1443790025251711791676013646288031456585152894843824089568509336042014947932956318658483844447958085527537280000762003935131376328099825405230904915558479367640186880000000000000000000000000000 ÷ 2571915383442530996565099826928437734823545228057475274989547702489888468035008300746456320383347822970443180013408465528945918134757657818850232249926961971855360000000000000000000000000000000)
  0  0   0.00% = 1 - (1 ÷ 1)
  2  0   0.00% = 1 - (1 ÷ 1)
  2  1   0.00% = 1 - (1 ÷ 1)
  4  0   0.00% = 1 - (6 ÷ 6)
  4  1   0.00% = 1 - (6 ÷ 6)
  4  2  33.33% = 1 - (4 ÷ 6)
  6  0   0.00% = 1 - (90 ÷ 90)
  6  1   0.00% = 1 - (90 ÷ 90)
  6  2  20.00% = 1 - (72 ÷ 90)
  6  3  60.00% = 1 - (36 ÷ 90)
  8  0   0.00% = 1 - (2520 ÷ 2520)
  8  1   0.00% = 1 - (2520 ÷ 2520)
  8  2  14.29% = 1 - (2160 ÷ 2520)
  8  3  42.86% = 1 - (1440 ÷ 2520)
  8  4  77.14% = 1 - (576 ÷ 2520)
 10  0   0.00% = 1 - (113400 ÷ 113400)
 10  1   0.00% = 1 - (113400 ÷ 113400)
 10  2  11.11% = 1 - (100800 ÷ 113400

 48 15  96.08% = 1 - (28997353519575134161840880733854879113543680000000000 ÷ 739927029164795438698666634999118747623055360000000000)
 48 16  97.86% = 1 - (15816738283404618633731389491193570425569280000000000 ÷ 739927029164795438698666634999118747623055360000000000)
 48 17  98.93% = 1 - (7908369141702309316865694745596785212784640000000000 ÷ 739927029164795438698666634999118747623055360000000000)
 48 18  99.52% = 1 - (3571521547865559046326442788334032031580160000000000 ÷ 739927029164795438698666634999118747623055360000000000)
 48 19  99.81% = 1 - (1428608619146223618530577115333612812632064000000000 ÷ 739927029164795438698666634999118747623055360000000000)
 48 20  99.93% = 1 - (492623661774559868458819694942625107804160000000000 ÷ 739927029164795438698666634999118747623055360000000000)
 48 21  99.98% = 1 - (140749617649874248131091341412178602229760000000000 ÷ 739927029164795438698666634999118747623055360000000000)
 48 22 100.00% = 1 - (31277692811083166251353631424928578273280000000

 94 41 100.00% = 1 - (23992712066091835085841798162162312153131065262127838856119812219749433582262530932503442494672242475008000000000000000000000 ÷ 772620123723827902234726289924442717479186651844591951796540559158509608077863558365756867477951285100067946496000000000000000000000)
 94 42 100.00% = 1 - (5432312165907585302454746753697127279954203455576114457989391445981003829946233418680024715774847352832000000000000000000000 ÷ 772620123723827902234726289924442717479186651844591951796540559158509608077863558365756867477951285100067946496000000000000000000000)
 94 43 100.00% = 1 - (1044675416520689481241297452634062938452731433764637395767190662688654582681967965130773983802855260160000000000000000000000 ÷ 772620123723827902234726289924442717479186651844591951796540559158509608077863558365756867477951285100067946496000000000000000000000)
 94 44 100.00% = 1 - (163870653571872859802556463158284382502389244512099983649755398068808561989328308255807683733781217280000000000000000000000 ÷ 77

In [2]:
#!/usr/bin/env python3
#
# testmail.py
#

import __main__, os, sys

print(hasattr(__main__, '__file__'))                # only False in Jupyter

if __name__ == '__main__':
    is_idle, is_pycharm, is_jupyter = (
        'idlelib' in sys.modules,
        int(os.getenv('PYCHARM', 0)),
        '__file__' not in globals()
        )
    if is_idle or is_pycharm or is_jupyter:
        print('IDE')
    else:
        print('import')


False
IDE
